# 网络与Web编程

## 作为客户端与HTTP服务交互

你需要通过HTTP协议以客户端的方式访问多种服务。例如，下载数据或者与基于REST的API进行交互

对于简单的事情来说，通常使用urllib.request模块就够了。例如，发送一个简单的HTTPGET请求到远程的服务上，可以这样做

In [3]:
from urllib import request, parse

url = 'http://httpbin.org/get'

parms = {
    'name1': 'value1',
    'name2': 'value2'
}

querystring = parse.urlencode(parms)

u = request.urlopen(url + '?' + querystring)
resp = u.read()

In [4]:
resp

b'{\n  "args": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7"\n  }, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/get?name1=value1&name2=value2"\n}\n'

如果你需要使用POST方法在请求主体中发送查询参数，可以将参数编码后作为可选参数提供给urlopen()函数，就像这样：

In [7]:
from urllib import request, parse

url = 'http://httpbin.org/post'
parms = {
    'name1': 'value1',
    'name2': 'value2'
}
querystring = parse.urlencode(parms, querystring.encode('ascii'))
resp = u.read()


In [8]:
resp

b''

如果你需要在发出的请求中提供一些自定义的HTTP头，例如修改user-agent字 段,可以创建一个包含字段值的字典，并创建一个Request实例然后将其传给urlopen()，如下：

In [10]:
from urllib import request, parse

headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
    'Spam': 'Eggs'
}

req = request.Request(url, querystring.encode('ascii'), headers=headers)

u = request.urlopen(req)
resp = u.read()

如果需要交互的服务比上面的例子都要复杂，也许应该去看看requests库（https://pypi.python.org/pypi/requests）。例如，下面这个示例采用requests库重新实现了上面的操作：

In [11]:
import requests

url = 'http://httpbin.org/post'
parms = {
    'name1': 'value1',
    'name2': 'value2'
}

headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
    'Spam': 'Eggs'
}

resp = requests.post(url, data=parms, headers=headers)
text = resp.text

关于requests库，一个值得一提的特性就是它能以多种方式从请求中返回响应结果的内容。从上面的代码来看，resp.text带给我们的是以Unicode解码的响应文本。但是，如果去访问resp.content，就会得到原始的二进制数据。另一方面，如果访问 
resp.json，那么就会得到JSON格式的响应内容。

下面这个示例利用requests库发起一个HEAD请求，并从响应中提取出一些HTTP头数据的字段：

In [12]:
text

'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0"\n  }, \n  "json": null, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/post"\n}\n'

In [13]:
resp.content

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0"\n  }, \n  "json": null, \n  "origin": "114.242.9.141, 114.242.9.141", \n  "url": "https://httpbin.org/post"\n}\n'

In [15]:
(resp.json)['args']

TypeError: 'method' object is not subscriptable

In [16]:
resp.json

<bound method Response.json of <Response [200]>>

下面这个示例利用requests库发起一个HEAD请求，并从响应中提取出一些HTTP头数据的字段：

In [20]:
import requests

resp = requests.head('http://www.python.org/index.html')

status = resp.status_code  # 301
last_modified = resp.headers['Last-Modified']  # Not Have
content_type = resp.headers['content-type']  # Not Have
content_length = resp.headers['content-length']  # Not Have
status, last_modified, content_type, content_length

KeyError: 'last-modified'

In [22]:
resp.headers

{'Server': 'Varnish', 'Retry-After': '0', 'Location': 'https://www.python.org/index.html', 'Content-Length': '0', 'Accept-Ranges': 'bytes', 'Date': 'Thu, 17 Oct 2019 09:33:20 GMT', 'Via': '1.1 varnish', 'Connection': 'close', 'X-Served-By': 'cache-hnd18739-HND', 'X-Cache': 'HIT', 'X-Cache-Hits': '0', 'X-Timer': 'S1571304800.227863,VS0,VE0', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains'}

.......

坦白说，所有的这些操作在requests库中都变得简单的多

在开发过程中测试HTTP客户端代码常常是很令人沮丧的，因为所有棘手的细节 问题都需要考虑（例如cookies、认证、HTTP头、编码方式等）。要完成这些任务，考虑使用httpbin服务（http://httpbin.org）。这个站点会接收发出的请求，然后以JSON 的形式将相应信息回传回来。下面是一个交互式的例子：

In [28]:
import requests

r = requests.get(
    'http://httpbin.org/get?name=Dave&n=37',
    headers = {'User-agent': 'goaway/1.0'}
)

resp = r.json() # 原文报错 r.json
resp['headers']

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'goaway/1.0'}

In [24]:
resp

<bound method Response.json of <Response [200]>>

In [25]:
r

<Response [200]>

In [26]:
r.json

<bound method Response.json of <Response [200]>>

In [29]:
resp['headers']

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'goaway/1.0'}

In [30]:
resp['args']

{'n': '37', 'name': 'Dave'}

在要同一个真正的站点进行交互前，先在httpbin.org这样的网站上做实验常常是可取的办法。尤其是当我们面对3次登录失败就会关闭账户这样的风险时尤为有用（不要尝试自己编写HTTP认证客户端来登录你的银行账户）。

尽管本节没有涉及，request库还对许多高级的HTTP客户端协议提供了支持，比如OAuth。requests模块的文档（http://docs.python-requests.org)质量很高（坦白说比在这短短的一节的篇幅中所提供的任何信息都好），可以参考文档以获得更多地信息。

## 创建TCP服务器

你想实现一个服务器，通过TCP协议和客户端通信

创建一个TCP服务器的一个简单方法是使用socketserver库。例如，下面是一个简单的应答服务器

In [ ]:
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler):
    def handle(self):  # 用来为客户端连接服务
        print('Got connection from', self.client_address)  # client_address有客户端地址
        while True:
            msg = self.request.recv(8192) # 8192?  request属性是客户端socket
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()

Got connection from ('127.0.0.1', 55603)


在这段代码中，你定义了一个特殊的处理类，实现了一个handle()方法，用来为客户端连接服务。request属性是客户端socket，client_address有客户端地址。为了测试这个服务器，运行它并打开另外一个Python进程连接这个服务器：

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

In [ ]:
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 20000))
s.send(b'hello')  #  5

In [ ]:
s.recv(8192)  # b'hello'

很多时候，可以很容易的定义一个不同的处理器。下面是一个使用StreamRequestHandler基类将一个类文件接口放置在底层socket上的例子：

In [ ]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print('Got connecttion from', self.client_address)
        for line in self.rfile:  # ?
            self.wfile.write(line)
            
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()

socketserver可以让我们很容易的创建简单的TCP服务器。但是，你需要注意的是，默认情况下这种服务器是单线程的，一次只能为一个客户端连接服务。如果你想处理多个客户端，可以初始化一个ForkingTCPServer或者是ThreadingTCPServer对象。例如：

In [ ]:
from socketserver import ThreadingTCPServer

if __name__ == '__main__':
    serv = ThreadingTCPServer(('', 20000), EchoHandler)
    serv.serve_forever()

使用fork或线程服务器有个潜在问题就是它们会为每个客户端连接创建一个新的进程或线程。由于客户端连接数是没有限制的，因此一个恶意的黑客可以同时发送大量 的连接让你的服务器奔溃。

如果你担心这个问题，你可以创建一个预先分配大小的工作线程池或进程池。你先 创建一个普通的非线程服务器，然后在一个线程池中使用serve_forever()方法来启动它们。

In [ ]:
from threading import Thread
from socketserver import TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print('Got connecttion from', self.client_address)
        while True:
            msg = self.request.recv(8192) # 8192?  request属性是客户端socket
            if not msg:
                break
            self.request.send(msg)

if __name__ == '__main__':
    nworkers = 16
    serv = TCPServer(('', 20000), EchoHandler)
    for n in range(nworkers):
        t = Thread(target=serv.serve_forever)
        t.daemon = True
        t.start()
    serv.serve_forever()
    
'''
一般来讲，一个TCPServer在实例化的时候会绑定并激活相应的socket 。
不过，有时候你想通过设置某些选项去调整底下的socket‘，
可以设置参数 bind_and_activate=False。如下：
'''
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler, bind_and_activate=False)
    serv.socket.sockopt() # 索引不出来
    serv.server_bind()
    serv.server_activate()
    serv.serve_forever()

最后，还需要注意的是巨大部分Python的高层网络模块（比如HTTP、XML-RPC 等）都是建立在socketserver功能之上。也就是说，直接使用socket库来实现服务器也并不是很难。下面是一个使用socket直接编程实现的一个服务器简单例子：

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

def echo_handler(address, client_sock):
    print('Got connection from {} '.format(address))
    while True:
        msg = client_sock.recv(8192)
        if not msg:
            break
        client_sock.sendall(msg)
    client_sock.close()
    
def echo_server(address, backlog=5):
    sock = socket(AF_INET, SOCK_STEARM)
    sock.bind(address)
    sock.listen(backlog)
    while True:
        client_sock, client_addr = sock.accept()
        echo_handler(client_addr, client_sock())
        
if __name__ == '__main__':
    echo_server(('', 20000))

## 创建UDP服务器

你想实现一个基于UDP协议的服务器来与客户端通信

跟TCP一样，UDP服务器也可以通过使用socketserver库很容易的被创建。例如，下面是一个简单的时间服务器：

In [ ]:
from socketserver import BaseRequestHandler, UDPServer
import time

class TimeHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        msg, sock = self.request
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), self.client_address)
        
if __name__ == '__main__':
    serv = UDPServer(('', 20000), TimeHandler)
    serv.serve_forever()


一个典型的UDP服务器接收到达的数据报(消息)和客户端地址。如果服务器需要做应答，它要给客户端回发一个数据报。对于数据报的传送，你应该使用socket的
sendto()和recvfrom()方法。尽管传统的send()和recv()也可以达到同样的效果，但是前面的两个方法对于UDP连接而言更普遍。

UDPServer类是单线程的，也就是说一次只能为一个客户端连接服务。实际使用中，这个无论是对于UDP还是TCP都不是什么大问题。如果你想要并发操作，可以实例化一个ForkingUDPServer或ThreadingUDPServer对象：

In [ ]:
from socketserver import ThreadingUDPServer

if __name__ == '__main__':
    serv = ThreadingUDPServer(('', 20000), TimeHandler)
    serv.serve_forever()

直接使用socket来实现一个UDP服务器也不难，下面是一个例子：

In [ ]:
from socket import socket, AF_INET, SOCK_DGRAM
import time

def time_server(address):
    sock = socket(AF_INET, SOCK_DGRAM)
    sock.bind(address)
    while True:
        msg, addr = sock.recvfrom(8192)
        pirnt('Got message from', addr)
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), addr)
        
if __nam__ == '__main__':
    time_server(('', 20000))

## 通过CIDR地址生成对应的IP地址集

你有一个CIDR网络地址比如“123.45.67.89/27”，你想将其转换成它所代表的所有IP（比如，“123.45.67.64”,“123.45.67.65”,…,“123.45.67.95”)）

可以使用ipaddress模块很容易的实现这样的计算。例如：

In [1]:
import ipaddress
net = ipaddress.ip_network('123.45.67.64/27')
net

IPv4Network('123.45.67.64/27')

In [2]:
for a in net:
    print(a) # 打印所在子网的地址

123.45.67.64
123.45.67.65
123.45.67.66
123.45.67.67
123.45.67.68
123.45.67.69
123.45.67.70
123.45.67.71
123.45.67.72
123.45.67.73
123.45.67.74
123.45.67.75
123.45.67.76
123.45.67.77
123.45.67.78
123.45.67.79
123.45.67.80
123.45.67.81
123.45.67.82
123.45.67.83
123.45.67.84
123.45.67.85
123.45.67.86
123.45.67.87
123.45.67.88
123.45.67.89
123.45.67.90
123.45.67.91
123.45.67.92
123.45.67.93
123.45.67.94
123.45.67.95


In [3]:
net6=ipaddress.ip_network('12:3456:78:90ab:cd:ef01:23:30/125')
net6

IPv6Network('12:3456:78:90ab:cd:ef01:23:30/125')

In [4]:
for a in net6:
    print(a)

12:3456:78:90ab:cd:ef01:23:30
12:3456:78:90ab:cd:ef01:23:31
12:3456:78:90ab:cd:ef01:23:32
12:3456:78:90ab:cd:ef01:23:33
12:3456:78:90ab:cd:ef01:23:34
12:3456:78:90ab:cd:ef01:23:35
12:3456:78:90ab:cd:ef01:23:36
12:3456:78:90ab:cd:ef01:23:37


In [7]:
net.num_addresses, net[0]

(32, IPv4Address('123.45.67.64'))

In [8]:
a = ipaddress.ip_address('123.45.67.81')
a in net

True

In [10]:
b = ipaddress.ip_address('123.45.67.181')
b in net

False

In [11]:
inet = ipaddress.ip_interface('123.45.67.73/27')
inet.network

IPv4Network('123.45.67.64/27')

In [12]:
inet.network

IPv4Network('123.45.67.64/27')

In [13]:
inet.ip

IPv4Address('123.45.67.73')

ipaddress模块有很多类可以表示IP地址、网络和接口。当你需要操作网络地址（比如解析、打印、验证等）的时候会很有用。

要注意的是，ipaddress模块跟其他一些和网络相关的模块比如socket库交集很少。所以，你不能使用IPv4Address的实例来代替一个地址字符串，你首先得显式的使用str()转换它。例如：

In [14]:
a = ipaddress.ip_address('127.0.0.1')
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.connect((a, 8000))

TypeError: str, bytes or bytearray expected, not IPv4Address

In [16]:
s.connect((str(a), 8080))

ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

## 创建一个简单的REST接口

你想使用一个简单的REST接口通过网络远程控制或访问你的应用程序，但是你又不想自己去安装一个完整的web框架。

构建一个REST风格的接口最简单的方法是创建一个基于WSGI标准（PEP 3333）的很小的库，下面是一个例子：

In [2]:
_hello_resp = '''
<html>
  <head>
    <title>Hello {name} </title>
  </head>
  <boby>
    <h1>Hello {name}</h1>
  </boby>
</html>'''

In [3]:
_localtime_resp = '''\
<?xml version="1.0"?>
<time>
  <year>{t.tm_year}</year>
  <month>{t.tm_mon}</month>
  <day>{t.tm_mday}</day>
  <hour>{t.tm_hour}</hour>
  <minute>{t.tm_min}</minute>
  <second>{t.tm_sec}</second>
</time>'''

In [ ]:
import cgi
import time
from wsgiref.simple_server import make_server

def notfound_404(environ, start_response):
    '''
    environ：一个包含所有HTTP请求信息的dict对象
    start_response：一个发送HTTP响应的函数
    HTTP响应的输出都可以通过start_response()加上函数返回值作为Body
    '''
    start_response('404 Not Found', [('Content-type', 'text/plain')])
    return [b'Not Found']

def hello_world(envrion, start_response):
    start_response('200 OK', [('Contetn-type', 'text/html')])
    params = envrion['params']
    resp = _hello_resp.format(name=params.get('name'))
    yield resp.encode('utf-8')

def localtime(environ, start_response):
    start_response('200 OK', [('Content-type', 'application/xml')])
    resp = _localtime_resp.format(t=time.localtime())
    yield resp.encode('utf-8')
    
class PathDispatcher:
    
    def __init__(self):
        self.pathmap = {}

    def __call__(self, environ, start_response):
        path = environ['PATH_INFO']
        params = cgi.FieldStorage(environ['wsgi.input'], environ=environ)
        method = environ['REQUEST_METHOD'].lower()
        environ['params'] = {key:params.getvalue(key) for key in params}
        
        handler = self.pathmap.get((method, path), notfound_404)  # 返回指定键的值，如果值不在字典中返回默认值
        return handler(environ, start_response)

    def register(self, method, path, function):
        self.pathmap[method.lower(), path] = function  # key--->(method.lower(), path) 
        return function

if __name__ == '__main__':
    dispatcher = PathDispatcher()
    dispatcher.register('GET', '/hello', hello_world)  # --->
    dispatcher.register('GET', '/localtime', localtime)

    httpd = make_server('', 8011, dispatcher)
    print('Serving on port 8011....')
    httpd.serve_forever()

在 WSGI 中，你可以像下面这样约定的方式以一个可调用对象形式来实现你的程序。

为了实现一个简单的 REST 接口，你只需让你的程序代码满足 Python 的 WSGI
标准即可。WSGI 被标准库支持，同时也被绝大部分第三方 web 框架支持。因此，如
果你的代码遵循这个标准，在后面的使用过程中就会更加的灵活！

In [ ]:
import cgi

def wsgi_app(environ, start_response):
    '''
    environ：type->dict 包含了从web服务器提取的CGI接口中获取的值
    start_response: 
    为了初始化一个请求对象而必须被调用的函数
    第一个参数返回HTTP状态值，第二个参数是一个(名，值)元组列表，用来构建返回http头
    
    return 为了返回数据，一个WSGI程序必须返回一个字节字符串序列
    '''
    pass

In [ ]:
import cgi

def wsgi_app(environ, start_response):
    method = environ['REQUEST_METHOD']
    path = environ['PATH_INFO']
    params = cgi.FieldStorage(environ['wsgi.input'], environ=environ)

In [ ]:
import cgi

def wsgi_app(environ, start_response):
    start_response('200 OK', [('Content-type', 'text/plain')])
    resp = []
    resp.append(b'Hello World\n')
    resp.append(b'Good Bye\n')
    return resp

或者，你可以使用yield

In [ ]:
import cgi

def wsgi_app(environ, start_response):
    start_response('200 OK', [('Content-type', 'text/plain')])
    yield b'Hello World\n'
    yield b'Good Bye\n'
    pass

这里要强调的一点是最后返回的必须是字节字符串。如果返回结果包含文本字符
串，必须先将其编码成字节。当然，并没有要求你返回的一定是文本，你可以很轻松的
编写一个生成图片的程序

尽管 WSGI 程序通常被定义成一个函数，不过你也可以使用类实例来实现，只要
它实现了合适的 __call__() 方法。例如

In [ ]:
class WSGIApplication:
    def __init__(self):
        pass
    
    def __call__(self, environ, start_response):
        pass

最后，使用 WSGI 还有一个很重要的部分就是没有什么地方是针对特定 web 服务
器的。因为标准对于服务器和框架是中立的，你可以将你的程序放入任何类型服务器
中。我们使用下面的代码测试测试本节代码

In [ ]:
if __name__ == '__main__':
    dispatcher = PathDispatcher()
    pass

    httpd = make_server('', 8080, dispatcher)
    print('Serving on port 8080')
    httpd.server_forever()

## 通过XML-PRC实现简单的远程调用

..............

## 在不同的Python解释器之间交互

..........................

## 简单的客户端认证

............................

## 理解时间驱动的IO

你应该已经听过基于事件驱动或异步 I/O 的包，但是你还不能完全理解它的底层
到底是怎样工作的，或者是如果使用它的话会对你的程序产生什么影响。

事件驱动 I/O 本质上来讲就是将基本 I/O 操作（比如读和写）转化为你程序需要
处理的事件。例如，当数据在某个 socket 上被接受后，它会转换成一个 receive 事件，
然后被你定义的回调方法或函数来处理。作为一个可能的起始点，一个事件驱动的框架
可能会以一个实现了一系列基本事件处理器方法的基类开始：

In [ ]:
class EventHandler:
    def fileno(self):
        raise NotImplemented('must implement'.title)
        
    def wants_to_receive(self):
        return False
    
    def handle_receive(self):
        pass
    
    def wants_to_send(self):
        return False
    
    def handle_send(self):
        pass

这个类的实例作为插件被放入类似下面这样的事件循环中

In [ ]:
import select

def event_loop(handlers):
    while True:
        wants_recv = [h for h in handlers if h.wants_to_receive()]
        wants_send = [h for h in handlers if h.wants_to_send()]
        can_recv, can_send, _ = select.select(wants_recv, wants_send, [])
        for h in can_recv:
            h.handle_receive()
        for h in can_send:
            h.handle_send()

...............

## 发送与接收大型数组

..................